In [158]:
import pandas as pd
import numpy as np
import re
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
import matplotlib.pyplot as plt
import seaborn as sns

In [159]:
# Cargar el dataset
df = pd.read_csv("https://raw.githubusercontent.com/shotokan/spam-classifier/refs/heads/main/data/sms.tsv", sep="\t", names=["label", "message"])
print(df)

     label                                            message
0      ham  Go until jurong point, crazy.. Available only ...
1      ham                      Ok lar... Joking wif u oni...
2     spam  Free entry in 2 a wkly comp to win FA Cup fina...
3      ham  U dun say so early hor... U c already then say...
4      ham  Nah I don't think he goes to usf, he lives aro...
...    ...                                                ...
5567  spam  This is the 2nd time we have tried 2 contact u...
5568   ham               Will ü b going to esplanade fr home?
5569   ham  Pity, * was in mood for that. So...any other s...
5570   ham  The guy did some bitching but I acted like i'd...
5571   ham                         Rofl. Its true to its name

[5572 rows x 2 columns]


In [160]:
# Cargar segundo dataset
temporal_df = pd.read_csv("https://raw.githubusercontent.com/shotokan/spam-classifier/refs/heads/main/data/ham-spam.csv")
second_df = temporal_df.rename(columns={
    "IsSpam": "label",
    "Text": "message"
})
second_df["label"] = second_df["label"].map({0: "ham", 1: "spam"})
print(second_df)

    label                                            message
0     ham  key issues going forwarda year end reviews rep...
1     ham  congrats contratulations the execution the cen...
2     ham   key issues going forwardall under control set...
3     ham  epmi files protest entergy transcoattached our...
4     ham  california power please contact kristin walsh ...
..    ...                                                ...
995  spam  somebody nort offlce pro offlce ado phot shop ...
996  spam   utf present day course utf reduce mass this p...
997  spam   sell regalis for affordable pricehi regalis a...
998  spam  email exclusive complimentary satellite dish w...
999  spam  unfeigned alilum ciall ambiien aagrra xaanax c...

[1000 rows x 2 columns]


In [161]:
# Mezclar los dataset para teber un dataset con mas datos
merged_df = pd.concat([df, second_df], ignore_index=True)
print(merged_df)

     label                                            message
0      ham  Go until jurong point, crazy.. Available only ...
1      ham                      Ok lar... Joking wif u oni...
2     spam  Free entry in 2 a wkly comp to win FA Cup fina...
3      ham  U dun say so early hor... U c already then say...
4      ham  Nah I don't think he goes to usf, he lives aro...
...    ...                                                ...
6567  spam  somebody nort offlce pro offlce ado phot shop ...
6568  spam   utf present day course utf reduce mass this p...
6569  spam   sell regalis for affordable pricehi regalis a...
6570  spam  email exclusive complimentary satellite dish w...
6571  spam  unfeigned alilum ciall ambiien aagrra xaanax c...

[6572 rows x 2 columns]


In [162]:
# Label como número
merged_df['label_num'] = merged_df['label'].map({'ham': 0, 'spam': 1})

# Feature: longitud del mensaje
merged_df['message_length'] = merged_df['message'].apply(len)

# Feature: cantidad de signos de puntuación
merged_df['punctuation_count'] = merged_df['message'].apply(lambda x: sum(1 for c in x if c in "!\"#$%&'()*+,-./:;<=>?@[\\]^_`{|}~"))

# Feature: proporción de mayúsculas
merged_df['uppercase_ratio'] = merged_df['message'].apply(lambda x: sum(1 for c in x if c.isupper()) / len(x) if len(x) > 0 else 0)

# Feature: densidad de puntuación
merged_df['punctuation_density'] = merged_df['punctuation_count'] / merged_df['message_length']

# Feature: contiene URL
merged_df['contains_url'] = merged_df['message'].str.contains(r'(?:http[s]?://|www\.)\S+', flags=re.IGNORECASE)

In [163]:
X_text = merged_df['message']
y = merged_df['label_num']

X_train_text, X_test_text, y_train, y_test = train_test_split(X_text, y, test_size=0.3, random_state=42)

In [155]:
# TF-IDF con unigrama + bigrama
vectorizer = TfidfVectorizer(ngram_range=(1, 2), stop_words='english')
X_train_vec = vectorizer.fit_transform(X_train_text)
X_test_vec = vectorizer.transform(X_test_text)

# Selección de features con chi2
selector = SelectKBest(score_func=chi2, k="all")
X_train_sel = selector.fit_transform(X_train_vec, y_train)
X_test_sel = selector.transform(X_test_vec)

chi2: Es una prueba estadística que mide la dependencia entre una variable independiente (una palabra, n-grama, etc.) y la variable objetivo (en este caso, spam o ham). Permite seleccionar solo las características más relevantes y descartar las irrelevantes, mejorando la eficiencia y precisión del modelo.

La prueba de Chi-cuadrado (χ²) es una herramienta estadística que mide si hay una dependencia entre dos variables categóricas.

En tu proyecto:

Una feature puede ser: que el mensaje contenga el trigram "win cash now" o la palabra "http".

La clase es: spam o ham.

La prueba de chi2 responde esta pregunta:

¿La presencia de esta palabra (o n-grama) está relacionada con que el mensaje sea spam?

Si la respuesta es sí, entonces esa palabra es informativa y se considera una buena feature.



In [169]:
model = MultinomialNB()
model.fit(X_train_sel, y_train)
y_pred = model.predict(X_test_sel)

In [157]:
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Matriz de confusión:\n", confusion_matrix(y_test, y_pred))
print("Reporte de clasificación:\n", classification_report(y_test, y_pred, target_names=['ham', 'spam']))

Accuracy: 0.9112576064908722
Matriz de confusión:
 [[1615    0]
 [ 175  182]]
Reporte de clasificación:
               precision    recall  f1-score   support

         ham       0.90      1.00      0.95      1615
        spam       1.00      0.51      0.68       357

    accuracy                           0.91      1972
   macro avg       0.95      0.75      0.81      1972
weighted avg       0.92      0.91      0.90      1972



In [164]:
# Con tri gramas
# Vectorizamos con trigramas únicamente
vectorizer_tri = TfidfVectorizer(ngram_range=(1, 3), stop_words='english')
X_train_tri = vectorizer_tri.fit_transform(X_train_text)
X_test_tri = vectorizer_tri.transform(X_test_text)

In [165]:
selector_tri = SelectKBest(score_func=chi2, k="all")  # puedes ajustar k según necesidad
X_train_tri_sel = selector_tri.fit_transform(X_train_tri, y_train)
X_test_tri_sel = selector_tri.transform(X_test_tri)

In [166]:

model_tri = MultinomialNB()
model_tri.fit(X_train_tri_sel, y_train)
y_pred_tri = model_tri.predict(X_test_tri_sel)

In [167]:
print("Accuracy:", accuracy_score(y_test, y_pred_tri))
print("Matriz de confusión:\n", confusion_matrix(y_test, y_pred))
print("Reporte de clasificación:\n", classification_report(y_test, y_pred_tri, target_names=["ham", "spam"]))

Accuracy: 0.9066937119675457
Matriz de confusión:
 [[1615    0]
 [ 175  182]]
Reporte de clasificación:
               precision    recall  f1-score   support

         ham       0.90      1.00      0.95      1615
        spam       1.00      0.48      0.65       357

    accuracy                           0.91      1972
   macro avg       0.95      0.74      0.80      1972
weighted avg       0.92      0.91      0.89      1972



El modelo de clasificación basado en texto y utilizando TF-IDF utilizando unigrama y bigrama, junto con selección de características mediante Chi-cuadrado conservando todas las variables (k="all"), logró el mejor rendimiento en las pruebas realizadas y comparando con tri-gramas. Con un accuracy del 91.1% y un F1-score para la clase spam de 0.68, demuestra un excelente balance entre precisión y capacidad de detección. Este resultado confirma que una combinación rica de n-gramas cortos y un filtrado basado en dependencia estadística mejora sustancialmente la capacidad del modelo para detectar mensajes no deseados. Aunque al modelo aún le falta para ser considerado como una base sólida para un sistema de detección de spam real.

Necesitamos mas datos para nuestro dataset, así como quizas incluir meta-features que nos puedan ayudar a una mejor clasificación.
Del mismo modo, con algún otro modelo como el uso de embeddings podriamos mejorarlo o con LogisticRegression.


In [168]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

# Entrenar modelo
log_model = LogisticRegression(max_iter=1000, class_weight='balanced')  # balanced para mejorar recall en spam
log_model.fit(X_train_sel, y_train)

# Predecir
y_pred_log = log_model.predict(X_test_sel)

# Evaluar
print("Accuracy:", accuracy_score(y_test, y_pred_log))
print("Matriz de confusión:\n", confusion_matrix(y_test, y_pred_log))
print("Reporte de clasificación:\n", classification_report(y_test, y_pred_log, target_names=["ham", "spam"]))

Accuracy: 0.9614604462474645
Matriz de confusión:
 [[1578   37]
 [  39  318]]
Reporte de clasificación:
               precision    recall  f1-score   support

         ham       0.98      0.98      0.98      1615
        spam       0.90      0.89      0.89       357

    accuracy                           0.96      1972
   macro avg       0.94      0.93      0.93      1972
weighted avg       0.96      0.96      0.96      1972



El modelo basado en regresión logística con penalización por clases (class_weight='balanced'), entrenado sobre vectores TF-IDF seleccionados con Chi², alcanzó un rendimiento sobresaliente:

Accuracy: 96.1%

Recall para spam: 89%

F1-score para spam: 0.89

Este modelo supera ampliamente a Naive Bayes, así como también logra una excelente capacidad de detección de mensajes no deseados con un bajo nivel de falsos positivos.